In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertModel, BertTokenizer, BertForSequenceClassification
from transformers import AdamW
import random
import copy

RANDOM_SEED = 42

torch.manual_seed(RANDOM_SEED)

In [2]:
def load_examples(file_path, do_lower_case=True):
    examples = []
    
    with open('{}/seq.in'.format(file_path),'r',encoding="utf-8") as f_text, open('{}/label'.format(file_path),'r',encoding="utf-8") as f_label:
        for text, label in zip(f_text, f_label):
            
            e = Inputexample(text.strip(),label=label.strip())
            examples.append(e)
            
    return examples

In [3]:
class Inputexample(object):
    def __init__(self,text_a,label = None):
        self.text = text_a
        self.label = label

In [4]:
N = 5
embed_dim = 768
batch_size = 32
lr= 0.001  # you can adjust 
temp = 0.3  # you can adjust 
lamda = 0.01  # you can adjust  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
path_5s = f'./BANKING77/train_5/'
path_test = f'./BANKING77/test/'
path_valid = f'./BANKING77/valid/'

In [6]:
train_samples = load_examples(path_5s)
print((train_samples[55].text))
print((train_samples[55].label))

where do i pay with my debit or credit card?
card_acceptance


In [7]:
class CustomTextDataset(Dataset):
    def __init__(self,labels,word_vector,batch_size,repeated_label:bool=False):
        self.labels = labels
        self.word_vector = word_vector
        self.batch_size = batch_size 
        self.count = 0 
        self.repeated_label = repeated_label

        if self.repeated_label == True:
            self.used_idx = []
          

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        label = self.labels[idx]
        
        data = self.word_vector[idx]
        
        sample = {"Class": label,"Text": data}
    
        return sample

In [8]:
from InferSent.models import InferSent
V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH)) 

<All keys matched successfully>

In [9]:
W2V_PATH = 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [10]:
# https://downgit.github.io/#/home?url=https:%2F%2Fgithub.com%2Fjianguoz%2FFew-Shot-Intent-Detection%2Ftree%2Fmain%2FDatasets%2FHWU64

# downloading training samples 
train_samples = load_examples(path_5s)
# write code here : for downloading validation samples
valid_samples = load_examples(path_valid)
#write code here : for downloading test samples
test_samples = load_examples(path_test)

# preprocess for training 
train_data = []
train_label = []
for i in range(len(train_samples)):
    train_data.append(train_samples[i].text)
    train_label.append(train_samples[i].label)
    
# write code here :preprocess validation samples
valid_data = []
valid_labels = []
for i in range(len(valid_samples)):
    valid_data.append(valid_samples[i].text)
    valid_labels.append(valid_samples[i].label)
# write code here : preprocess test samples
test_data = []
test_labels = []
for i in range(len(valid_samples)):
    test_data.append(test_samples[i].text)
    test_labels.append(test_samples[i].label)

In [11]:
infersent.build_vocab_k_words(5e5)

Vocab size : 500000.0


In [12]:
# dataloader for training 
train_data = CustomTextDataset(train_label,train_data,batch_size=batch_size,repeated_label=True)
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)

# write code here : dataloader for validation
valid_data = CustomTextDataset(valid_labels,valid_data,batch_size=batch_size,repeated_label=True)
valid_loader = DataLoader(valid_data,batch_size=batch_size,shuffle=True)

# write code here : dataloader for test
test_data = CustomTextDataset(test_labels,test_data,batch_size=batch_size,repeated_label=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=True)

# got the number of unique classes from dataset
num_class = len(np.unique(np.array(train_label)))

# get text label of uniqure classes
unique_label = np.unique(np.array(train_label))

# map text label to index classes
label_maps = {unique_label[i]: i for i in range(len(unique_label))}

In [13]:
class InferSentClassification(nn.Module):
    def __init__(self):
        super(InferSentClassification, self).__init__()
        self.fc1 = nn.Linear(4096, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 77)
        self.relu = nn.ReLU()
    
    def forward(self, input):
        x = self.fc1(input)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [14]:
model = InferSentClassification()

In [15]:
optimizer= torch.optim.AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [16]:
num_epochs = 50

In [17]:
# # freeze encoder layer
# for name, param in model.named_parameters():
#     if 'InferSent' in name:
#         param.requires_grad = False

In [18]:
from tqdm.auto import tqdm

model.cuda()
for epoch in range(num_epochs):

    model.train()
    train_loss = 0.0
    train_correct = 0.0
    for batch in train_loader:
        optimizer.zero_grad()

        sentences = batch["Text"]
        sentence_vectors = torch.tensor(infersent.encode(sentences, bsize=128, tokenize=True, verbose=False))
        labels = torch.tensor([label_maps[stringtoId] for stringtoId in (batch['Class'])])

        sentence_vectors = sentence_vectors.to(device)
        labels = labels.to(device)

        outputs = model(sentence_vectors)
        loss = criterion(outputs, labels)
        predicted = torch.argmax(outputs, dim=1)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_correct += torch.sum(predicted == labels.data)
    train_acc = train_correct / len(train_loader.dataset)
    print(f'Training loss {train_loss} training accuracy {train_acc}')

    if epoch % 5 == 0:
        model.eval()
        eval_correct = 0.0
        best_val_acc = torch.inf
        for batch in valid_loader:
            sentences = batch["Text"]
            sentence_vectors = torch.tensor(infersent.encode(sentences, bsize=128, tokenize=True, verbose=False))
            labels = torch.tensor([label_maps[stringtoId] for stringtoId in (batch['Class'])])
            
            sentence_vectors = sentence_vectors.to(device)
            labels = labels.to(device)

            outputs = model(sentence_vectors)
            predicted = torch.argmax(outputs, dim=1)
            eval_correct += torch.sum(predicted == labels.data)
        eval_acc = eval_correct / len(valid_loader.dataset)
        print(f'Validation acc {eval_acc}')
        if eval_acc < best_val_acc:
            best_val_acc = eval_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), "infersent_calassification_10shot_new.pth")

/home/pasit/Documents/NLP-project/InferSent/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


Training loss 56.613510608673096 training accuracy 0.015584414824843407
Validation acc 0.02207792177796364
Training loss 55.84168481826782 training accuracy 0.04935064911842346
Training loss 54.990524768829346 training accuracy 0.08311688154935837
Training loss 53.73274087905884 training accuracy 0.12467531859874725
Training loss 52.86895489692688 training accuracy 0.15844155848026276
Training loss 51.19095849990845 training accuracy 0.20519480109214783
Validation acc 0.20064935088157654
Training loss 50.0514030456543 training accuracy 0.26753246784210205
Training loss 47.68717050552368 training accuracy 0.26493504643440247
Training loss 46.33394527435303 training accuracy 0.3194805085659027
Training loss 44.95954966545105 training accuracy 0.350649356842041
Training loss 42.83592891693115 training accuracy 0.41038960218429565
Validation acc 0.2889610230922699
Training loss 41.913785219192505 training accuracy 0.4311688244342804
Training loss 40.14120125770569 training accuracy 0.42077

In [19]:
best_val_acc

tensor(0.5494, device='cuda:0')